<a href="https://colab.research.google.com/github/weiwei20060505/Jules-Art-Station-Generator/blob/main/%E5%84%AA%E5%8C%96%E7%9A%84_ComfyUI_Colab_%E5%95%9F%E5%8B%95%E7%AD%86%E8%A8%98%E6%9C%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title ComfyUI 超速啟動筆記本 (完整優化版)
#@markdown ## 🚀 第一步：安裝與快取
#@markdown 這個儲存格會執行所有安裝、下載與快取任務。所有檔案都會被儲存到你的 Google Drive，第一次執行會稍久，但之後每次啟動都將極速完成！
#@markdown ---

import os
import time
from google.colab import drive

# --- 核心優化 1: 掛載 Google Drive 作為永久快取儲存 ---
# 這是所有優化的基礎，將 Drive 視為一個持久化的硬碟。
print("🔄 [1/7] 掛載 Google Drive...")
drive.mount('/content/drive')
print("✅ Google Drive 掛載完成！")

# --- 定義所有路徑，將工作目錄指向 Drive ---
working_dir = '/content/drive/MyDrive/ComfyUI_Optimized'
comfyui_path = os.path.join(working_dir, 'ComfyUI')
custom_nodes_path = os.path.join(comfyui_path, 'custom_nodes')
models_path = os.path.join(comfyui_path, 'models')

# 建立基礎目錄結構
os.makedirs(os.path.join(models_path, 'checkpoints'), exist_ok=True)
os.makedirs(os.path.join(models_path, 'controlnet'), exist_ok=True)
os.makedirs(custom_nodes_path, exist_ok=True)

start_time = time.time()

# --- 核心優化 2: 條件性安裝 (Conditional Installation) ---
# 透過 `os.path.exists()` 檢查檔案是否存在，實現快取邏輯。
# if (exists) -> git pull (增量更新) else -> git clone (完整下載)
print("\n🔄 [2/7] 檢查並安裝 ComfyUI 主程式...")
if os.path.exists(comfyui_path):
    print("  -> ComfyUI 已存在，執行快速更新...")
    %cd {comfyui_path}
    !git pull
else:
    print("  -> ComfyUI 不存在，執行全新安裝...")
    %cd {working_dir}
    !git clone https://github.com/comfyanonymous/ComfyUI.git
print("✅ ComfyUI 準備就緒！")

print("\n🔄 [3/7] 檢查並安裝 ComfyUI-Manager...")
manager_path = os.path.join(custom_nodes_path, 'ComfyUI-Manager')
if os.path.exists(manager_path):
    print("  -> ComfyUI-Manager 已存在，執行快速更新...")
    %cd {manager_path}
    !git pull
else:
    print("  -> 安裝 ComfyUI-Manager...")
    %cd {custom_nodes_path}
    !git clone https://github.com/ltdrdata/ComfyUI-Manager.git
print("✅ ComfyUI-Manager 準備就緒！")

# --- 核心優化 3: 批次處理依賴 (Batch Processing) ---
print("\n🔄 [4/7] 安裝 Python 依賴套件...")
%cd {comfyui_path}
!pip install -r requirements.txt --extra-index-url https://download.pytorch.org/whl/cu121 -q
print("✅ Python 依賴安裝完成！")

# --- 核心優化 4: 並行下載 (Parallel Downloading) ---
# 使用 aria2c 多線程下載工具，大幅提升大檔案下載速度。
print("\n🔄 [5/7] 安裝 aria2c 加速器...")
!apt-get -y install -qq aria2c
print("✅ aria2c 安裝完成！")

print("\n🔄 [6/7] 檢查並下載 ControlNet 模型...")
controlnet_models = {
    "control_v11p_sd15_canny.pth": "https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_canny.pth",
    "control_v11p_sd15_openpose.pth": "https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_openpose.pth"
}
controlnet_path = os.path.join(models_path, 'controlnet')
for name, url in controlnet_models.items():
    file_path = os.path.join(controlnet_path, name)
    if not os.path.exists(file_path):
        print(f"  -> 正在下載 {name}...")
        !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M "{url}" -d "{controlnet_path}" -o "{name}"
    else:
        print(f"  -> 模型 {name} 已存在，跳過。")
print("✅ ControlNet 模型準備就緒！")

print("\n🔄 [7/7] 檢查並下載主模型 (Checkpoint)...")
main_model_url = "https://huggingface.co/runwayml/stable-diffusion-v1-5/resolve/main/v1-5-pruned-emaonly.ckpt"
main_model_name = "v1-5-pruned-emaonly.ckpt"
main_model_path = os.path.join(models_path, 'checkpoints', main_model_name)
if not os.path.exists(main_model_path):
    print(f"  -> 正在下載主模型 {main_model_name}...")
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M "{main_model_url}" -d "{os.path.join(models_path, 'checkpoints')}" -o "{main_model_name}"
else:
    print(f"  -> 主模型 {main_model_name} 已存在，跳過。")
print("✅ 主模型準備就緒！")

# --- 結算 ---
total_time = time.time() - start_time
print("\n" + "="*50)
print(f"🎉 所有依賴與模型均已準備就緒！")
print(f"⏱️ 本次安裝與檢查總耗時: {total_time:.2f} 秒")
print("="*50)



🔄 [1/7] 掛載 Google Drive...
Mounted at /content/drive
✅ Google Drive 掛載完成！

🔄 [2/7] 檢查並安裝 ComfyUI 主程式...
  -> ComfyUI 已存在，執行快速更新...
/content/drive/MyDrive/ComfyUI_Optimized/ComfyUI
You are not currently on a branch.
Please specify which branch you want to merge with.
See git-pull(1) for details.

    git pull <remote> <branch>

✅ ComfyUI 準備就緒！

🔄 [3/7] 檢查並安裝 ComfyUI-Manager...
  -> ComfyUI-Manager 已存在，執行快速更新...
/content/drive/MyDrive/ComfyUI_Optimized/ComfyUI/custom_nodes/ComfyUI-Manager
Updating 76ee93c9..eab6cdee
error: Your local changes to the following files would be overwritten by merge:
	custom-node-list.json
Please commit your changes or stash them before you merge.
Aborting
✅ ComfyUI-Manager 準備就緒！

🔄 [4/7] 安裝 Python 依賴套件...
/content/drive/MyDrive/ComfyUI_Optimized/ComfyUI
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188

In [2]:
#@title ## 🔥 第二步：啟動 ComfyUI
#@markdown ---
#@markdown ### ⚙️ 選擇你的硬體模式和網路隧道
#@markdown - **硬體模式**: 選擇 `GPU` 進行繪圖，選擇 `CPU` 僅用於整理工作流以節省資源。
#@markdown - **網路隧道**: 強烈推薦 `Cloudflared`，它沒有時間限制且穩定。
#@markdown ---
device = 'CPU' #@param ["GPU", "CPU"]
tunnel_service = 'Cloudflared' #@param ["Cloudflared", "Ngrok"]

# --- 核心優化 5: 條件啟動 (Conditional Launch) ---
# 根據使用者選擇的參數，動態構建啟動指令。
launch_command = f"python main.py --listen"
if device == 'CPU':
    launch_command += " --cpu"

# 根據選擇的隧道服務，在背景啟動它
if tunnel_service == 'Cloudflared':
    print("🚀 正在啟動 Cloudflared 隧道...")
    !pkill cloudflared
    !nohup cloudflared tunnel --url http://127.0.0.1:8188 > /content/cloudflared.log 2>&1 &
    time.sleep(4) # 等待 cloudflared 啟動並生成日誌
    print("🔗 你的 ComfyUI 連結是 (如果沒出現請稍等幾秒再重新執行此儲存格):")
    !grep -o 'https://[a-zA-Z0-9-]*\.trycloudflare.com' /content/cloudflared.log
    print("✅ Cloudflared 啟動成功！請點擊上面的連結。")
else: # Ngrok
    print("🚀 正在安裝與啟動 Ngrok 隧道...")
    !pip install pyngrok -q
    from pyngrok import ngrok
    ngrok.kill()
    public_url = ngrok.connect(8188)
    print(f"🔗 你的 ComfyUI 連結是: {public_url}")
    print("✅ Ngrok 啟動成功！(注意：免費版有2小時連線限制)")

# 進入 ComfyUI 目錄並在背景啟動主程式
%cd {comfyui_path}
print(f"\n🔥 正在以 {device} 模式啟動 ComfyUI 服務...")
print("🟢 服務啟動後，此儲存格會持續運行，請勿中斷。")
!{launch_command}

🚀 正在啟動 Cloudflared 隧道...
🔗 你的 ComfyUI 連結是 (如果沒出現請稍等幾秒再重新執行此儲存格):
✅ Cloudflared 啟動成功！請點擊上面的連結。
/content/drive/MyDrive/ComfyUI_Optimized/ComfyUI

🔥 正在以 CPU 模式啟動 ComfyUI 服務...
🟢 服務啟動後，此儲存格會持續運行，請勿中斷。
[START] Security scan
[DONE] Security scan
## ComfyUI-Manager: installing dependencies done.
** ComfyUI startup time: 2025-08-11 12:20:19.427
** Platform: Linux
** Python version: 3.11.13 (main, Jun  4 2025, 08:57:29) [GCC 11.4.0]
** Python executable: /usr/bin/python3
** ComfyUI Path: /content/drive/MyDrive/ComfyUI_Optimized/ComfyUI
** ComfyUI Base Folder Path: /content/drive/MyDrive/ComfyUI_Optimized/ComfyUI
** User directory: /content/drive/MyDrive/ComfyUI_Optimized/ComfyUI/user
** ComfyUI-Manager config path: /content/drive/MyDrive/ComfyUI_Optimized/ComfyUI/user/default/ComfyUI-Manager/config.ini
** Log path: /content/drive/MyDrive/ComfyUI_Optimized/ComfyUI/user/comfyui.log

Prestartup times for custom nodes:
   0.5 seconds: /content/drive/MyDrive/ComfyUI_Optimized/ComfyUI/custom_nodes/co